# Building the polarimetric data reduction from Q Tutorial
_written by William Melby_

A traditional dual-rotating retarder polarimeter (DRRP) works by generating different polarization states and measuring a linearly polarized output state (for more information, see FullMuellerExample.ipynb). This is an effective and widely used technique that gives information on the entire Mueller matrix of a sample. However, there is a way to get the same result using a slightly more robust method that uses measurements of the Q Stokes parameter.

Q is defined as the difference between the horizontal and vertical polarizations. As opposed to measuring a single linear polarization, this measurement takes information from a greater sampling of polarization states and is therefore more thorough. This measurement can be made using a Wollaston prism, which conveniently splits light into two beams with orthogonal polarization. In a traditional DRRP configuration, the last linear polarizer in the polarization state analyzer (PSA) can be replaced with a Wollaston prism to allow for Q sampling. 

The data taking steps for a Q-measurement system are exactly the same as for a standard DRRP, but the data reduction process is slightly different. By using a Wollaston prism, we get a direct sampling of Q or alternatively the total intensity by subtracting or adding the two orthogonal beams. However, this measurement doesn't change the polarization state for our purposes, so the Wollaston can be removed from the PSA model. The Mueller matrix representation for the system looks like:

$$\textbf{Q}_{meas} = [0, 1, 0, 0] * [\textbf{PSA}] * \textbf{M}_{sample} * [\textbf{PSG}] * \textbf{S}_{in}$$

where the PSG is the same as before but the PSA is just the Mueller matrix for a quarter-wave plate. 

From here, the data reduction process requires a combination of inputs from Q and the total intensity, I. If we only use one, we lose information about certain parts of the Mueller matrix. Specifically, measurements of I are accurate for the first row of the matrix and measurements of Q are good for the last three rows, so we use this combination of inputs to reconstruct the entire matrix. Thanfully, both Q and I can be found from a single set of images, so this additional requirement simplifies to just an additional data reduction calculation. Let's see how effective this is with a simulated matrix.

In [1]:
import numpy as np
import os
os.chdir("../../")
os.getcwd()
from katsu.polarimetry import *

In [27]:
M_rand = np.random.uniform(-1, 1, (4, 4))
print(M_rand)

[[ 0.12404759 -0.12263503  0.78603155 -0.54467126]
 [-0.09403614  0.58496692 -0.46455959  0.68697291]
 [-0.2826668  -0.36190585 -0.63223294 -0.01585697]
 [-0.94347673  0.93858152  0.483513   -0.92810663]]


In [3]:
theta = np.linspace(0, np.pi, 37) # angles at which the first quarter-wave plate will be rotated

Given this matrix and angles, let's simulate what the data would look like using a Wollaston prism to create two orthogonal polarizations. We will have two arrays, one for the vertical polarization and one for the horizontal, where each value corresponds to a measurment at a different angle. These arrays will be inputs for the data reduction functions that add and subtract them to find Q and the total I.

In [49]:
# # Lets simulate what the data would look like using this matrix, assuming no offsets in the other components
# hor_sim = np.array(single_output_simulation_function(theta, 0, 0, 0, 0, 0, 0, 0, M_rand)) # double check this
# vert_sim = np.array(single_output_simulation_function(theta, 0, 0, 0, 0, 0, 0, np.pi/2, M_rand))

In [61]:
# Now feed this data into a data reduction function to see how well the matrix correlates
M = q_calibrated_full_mueller_polarimetry(theta, 0, 0, 0, 0, 0, 2, 2, M_rand)
print(M)

[[ 0.12404759 -0.12263503  0.78603155 -0.54467126]
 [-0.09403614  0.58496692 -0.46455959  0.68697291]
 [-0.2826668  -0.36190585 -0.63223294 -0.01585697]
 [-0.94347673  0.93858152  0.483513   -0.92810663]]
